In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack


In [3]:
dr25Koi = pd.read_csv("../GKBaseline/koiCatalogs/dr25_kois_archive.txt", dtype={"kepoi_name":str})
print("Loaded " + str(len(dr25Koi)) + " KOIs")

fgkPCs = pd.read_csv("koiCatalogs/PCs_dr25_FGK.csv", dtype={"kepoi_name":str})
print("Loaded " + str(len(fgkPCs)) + " FGK PCs")


Loaded 8054 KOIs
Loaded 2520 FGK PCs


In [6]:
travisData = pd.read_csv("../data/GKSPCIIPapTable1_2020-05-28.csv")
for i in range(len(travisData)):
    travisData.loc[i, "kepoi_name"] = "K%08.2f"%travisData.loc[i].KOI
    
dr25Koi = pd.merge(dr25Koi, travisData, on="kepoi_name", how="inner")
dr25Koi["insolation"] = dr25Koi.gaia_iso_insol;


In [23]:
dr25KOIsInBox = dr25Koi[(dr25Koi.gaia_iso_prad < 2.5) 
                        & (dr25Koi.gaia_iso_insol < 2.2) 
                        & (dr25Koi.gaia_iso_insol > 0.2)
                        & (dr25Koi.koi_pdisposition == "CANDIDATE")]
print(str(len(dr25KOIsInBox)) + " KOIs in the box")

fgkPCsInBox = fgkPCs[(fgkPCs.gaia_iso_prad < 2.5) 
                        & (fgkPCs.gaia_iso_insol < 2.2) 
                        & (fgkPCs.gaia_iso_insol > 0.2)]
print(str(len(fgkPCsInBox)) + " FGKs in the box")


57 KOIs in the box
54 FGKs in the box


In [24]:
dr25KOIsInBoxNotInFgks = dr25KOIsInBox[~dr25KOIsInBox.kepid.isin(fgkPCsInBox.kepid_x)]
print(str(len(dr25KOIsInBoxNotInFgks)) + " KOIs in the box not in FGK")


3 KOIs in the box not in FGK


In [27]:
dr25KOIsInBoxNotInFgks

,kepid,kepoi_name,koi_tce_plnt_num,koi_pdisposition,koi_score,koi_period,koi_max_mult_ev,koi_prad,koi_prad_err1,koi_prad_err2,...,gaia_iso_prad_errm,gaia_iso_sma,gaia_iso_sma_errp,gaia_iso_sma_errm,gaia_iso_insol,gaia_iso_insol_errp,gaia_iso_insol_errm,ZAMSFlux,Flag,insolation
200,6435936,K00854.01,1,CANDIDATE,0.996,56.056075,19.254200,1.94,0.12,-0.22,...,0.083025,0.233494,0.001885,0.001954,0.944913,0.096387,0.084826,0.742053,HZAO,0.944913
2758,4172805,K04427.01,1,CANDIDATE,0.969,147.661729,10.793761,1.59,0.12,-0.14,...,0.094362,0.447414,0.003522,0.003619,0.318758,0.033427,0.030053,0.248065,HZAO,0.318758
3295,9278575,K07932.01,1,CANDIDATE,0.230,502.256480,7.206314,1.03,0.22,-0.15,...,0.235062,1.253948,0.027281,0.024871,2.033609,0.200840,0.213520,0.667615,NaN,2.033609


In [29]:
allStars = pd.read_csv("../stellarCatalogs/dr25_stellar_berger2020.txt")
print("Loaded " + str(len(allStars)) + " stars")


Loaded 162219 stars


In [64]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(p.kepoi_name, allStars[allStars.kepid == p.kepid].teff)

K00854.01 95088    3686.075212
Name: teff, dtype: float64
K04427.01 59962    3895.177497
Name: teff, dtype: float64
K07932.01 146747    5690.986001
Name: teff, dtype: float64


In [ ]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].RUWE)

In [42]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    dutyCycleChange = allStars[allStars.kepid == p.kepid].dutycycle - allStars[allStars.kepid == p.kepid].dutycycle_post
    print(dutyCycleChange)

95088    0.0095
dtype: float64
59962    0.0039
dtype: float64
146747    0.0036
dtype: float64


In [43]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].dutycycle)

95088    0.8732
Name: dutycycle, dtype: float64
59962    0.698
Name: dutycycle, dtype: float64
146747    0.8734
Name: dutycycle, dtype: float64


In [44]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].dataspan)

95088    1458.931
Name: dataspan, dtype: float64
59962    1458.931
Name: dataspan, dtype: float64
146747    1458.931
Name: dataspan, dtype: float64


In [45]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].timeoutsumry)

95088    1.0
Name: timeoutsumry, dtype: float64
59962    1.0
Name: timeoutsumry, dtype: float64
146747    1.0
Name: timeoutsumry, dtype: float64


In [46]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].limbdark_coeff1)
    print(allStars[allStars.kepid == p.kepid].limbdark_coeff2)
    print(allStars[allStars.kepid == p.kepid].limbdark_coeff3)
    print(allStars[allStars.kepid == p.kepid].limbdark_coeff4)
    
    
    

95088    0.609892
Name: limbdark_coeff1, dtype: float64
95088   -0.16199
Name: limbdark_coeff2, dtype: float64
95088    0.657235
Name: limbdark_coeff3, dtype: float64
95088   -0.263529
Name: limbdark_coeff4, dtype: float64
59962    0.585469
Name: limbdark_coeff1, dtype: float64
59962    0.163967
Name: limbdark_coeff2, dtype: float64
59962    0.173391
Name: limbdark_coeff3, dtype: float64
59962   -0.086793
Name: limbdark_coeff4, dtype: float64
146747    0.432897
Name: limbdark_coeff1, dtype: float64
146747    0.302951
Name: limbdark_coeff2, dtype: float64
146747    0.150653
Name: limbdark_coeff3, dtype: float64
146747   -0.147388
Name: limbdark_coeff4, dtype: float64


In [48]:
noisyTargets = pd.read_csv("../data/DR25_DEModel_NoisyTargetList.txt", header=9)
# cleanDr25GaiaStellar = cleanDr25GaiaStellar[~cleanDr25GaiaStellar.kepid.isin(noisyTargets['# 1- Kepler ID'])]
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].kepid.isin(noisyTargets['# 1- Kepler ID']))


95088    False
Name: kepid, dtype: bool
59962    False
Name: kepid, dtype: bool
146747    False
Name: kepid, dtype: bool


In [49]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(travisData[travisData.kepoi_name == p.kepoi_name].kepoi_name)


840    K00854.01
Name: kepoi_name, dtype: object
2950    K04427.01
Name: kepoi_name, dtype: object
3832    K07932.01
Name: kepoi_name, dtype: object


In [50]:
dr25KOIsInBoxNotInFgks.kepoi_name

200     K00854.01
2758    K04427.01
3295    K07932.01
Name: kepoi_name, dtype: object

In [51]:
dr25KOIsInBoxNotInFgks.insolation

200     0.944913
2758    0.318758
3295    2.033609
Name: insolation, dtype: float64

In [54]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(fgkPCs[fgkPCs.kepoi_name == p.kepoi_name].kepoi_name)


Series([], Name: kepoi_name, dtype: object)
Series([], Name: kepoi_name, dtype: object)
Series([], Name: kepoi_name, dtype: object)


In [55]:
cleanFGKs = pd.read_csv("../stellarCatalogs/dr25_stellar_berger2020_clean_FGK.txt")
print("Loaded " + str(len(cleanFGKs)) + " clean FGKs")


Loaded 80929 clean FGKs


In [58]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(cleanFGKs[cleanFGKs.kepid == p.kepid].kepid)

Series([], Name: kepid, dtype: int64)
Series([], Name: kepid, dtype: int64)
Series([], Name: kepid, dtype: int64)


In [57]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].Bin)

95088    0
Name: Bin, dtype: int64
59962    0
Name: Bin, dtype: int64
146747    0
Name: Bin, dtype: int64


In [59]:
evolvedStars = pd.read_csv("../stellarCatalogs/dr25_stellar_berger2020_evolved.txt")
print("Loaded " + str(len(cleanFGKs)) + " evolved stars")


Loaded 80929 evolved stars


In [60]:
for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(evolvedStars[evolvedStars.kepid == p.kepid].kepid)

Series([], Name: kepid, dtype: int64)
Series([], Name: kepid, dtype: int64)
50550    9278575
Name: kepid, dtype: int64


In [62]:

for i, p in dr25KOIsInBoxNotInFgks.iterrows():
    print(allStars[allStars.kepid == p.kepid].kepid, allStars[allStars.kepid == p.kepid].iso_gof)

95088    6435936
Name: kepid, dtype: int64 95088    0.997091
Name: iso_gof, dtype: float64
59962    4172805
Name: kepid, dtype: int64 59962    1.0
Name: iso_gof, dtype: float64
146747    9278575
Name: kepid, dtype: int64 146747    1.0
Name: iso_gof, dtype: float64
